In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/NLP_and_CV_Projects/NLP/Hung')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Jan 12 09:59:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 12.6MB/s 
     |████████████████████████████████| 2.9MB 50.7MB/s 
     |████████████████████████████████| 890kB 44.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=527a1cfb2432947ec58bb98bf7663e3e2518256a63bed01de6995a793eda782b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

#phobert = AutoModel.from_pretrained("vinai/phobert-base")
custokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
custokenizer.add_tokens('\n')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


1

In [ ]:
fake_path = "fake.txt"
real_path = "all_data.txt"

In [ ]:
def build_text_files(data, dest_path):
    f = open(dest_path, 'w')
    data_write = ''
    for poem in data:
        data_write += poem + "\n\n"
    data_write = re.sub("\n\n+","\n\n", data_write)
    f.write(data_write)

In [ ]:
import re
import pandas as pd

with open(fake_path) as f:
    fake = f.read()
fake = re.sub("\n3+","\n\n", fake)
fake = re.sub("\n \n","\n\n", fake)
fake = fake.split("\n\n")

with open(real_path) as f:
    real = f.read()
real = re.sub("\n3+","\n\n", real)
real = re.sub("\n \n","\n\n", real)
real = real.split("\n\n")

In [ ]:
import re
import pandas as pd

from sklearn.model_selection import train_test_split
train_fake, test_fake = train_test_split(fake,test_size=0.1) 
train_real, test_real = train_test_split(real,test_size=0.1) 

build_text_files(train_fake,'train_fake.txt')
build_text_files(test_fake,'test_fake.txt')
build_text_files(train_real,'train_real.txt')
build_text_files(test_real,'test_real.txt')

# print("Train dataset length: "+str(len(train)))
# print("Test dataset length: "+ str(len(test)))

,label,text
0,0,sài gòn cơn sốt giữa lòng \nhành vi dịch nạn n...
1,1,đường thu vàng cả thời gian \ngió se se lạnh v...


In [ ]:
# import os
# import torch
# from torch.utils.data.dataset import Dataset
# from transformers.tokenization_utils import PreTrainedTokenizer
# from filelock import FileLock
# from transformers.utils import logging
# from typing import Dict, List, Optional
# import pickle
# import random
# import time

# logger = logging.get_logger(__name__)
# class CusTextDataset(Dataset):
#     """
#     This will be superseded by a framework-agnostic approach
#     soon.
#     """

#     def __init__(
#         self,
#         tokenizer: PreTrainedTokenizer,
#         file_path: str,
#         block_size: int,
#         overwrite_cache=True,
#         cache_dir: Optional[str] = None,
#     ):
#         assert os.path.isfile(file_path), f"Input file path {file_path} not found"

#         block_size = block_size - tokenizer.num_special_tokens_to_add(pair=False)

#         directory, filename = os.path.split(file_path)
#         cached_features_file = os.path.join(
#             cache_dir if cache_dir is not None else directory,
#             "cached_lm_{}_{}_{}".format(
#                 tokenizer.__class__.__name__,
#                 str(block_size),
#                 filename,
#             ),
#         )

#         # Make sure only the first process in distributed training processes the dataset,
#         # and the others will use the cache.
#         lock_path = cached_features_file + ".lock"
#         with FileLock(lock_path):
#             logger.info(f"Creating features from dataset file at {directory}")

#             self.examples = []
#             self.masks = []
#             with open(file_path, encoding="utf-8") as f:
#                 text = f.read()

#             tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
#             # tokenized_text = tokenizer.encode_plus(text, padding=True, return_token_type_ids=False)['input_ids']

#             # for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
#             i = 0
#             while i < len(tokenized_text) - block_size + 1:
#                 inds = tokenized_text[i : i + block_size]
#                 for j in range(0, len(inds)):
#                      if inds[j]==64001:
#                          inds = inds[j+1:] #remove the first \n
#                          break
#                 for j in range(len(inds)-1, 0, -1):
#                      if inds[j]==64001:
#                          inds = inds[:j] #remove \n
#                          break
#                 while len(inds) < block_size:
#                     inds.append(0)
#                 i += len(inds)
#                 block = tokenizer.build_inputs_with_special_tokens(inds)
#                 mask = [int(token_id > 0) for token_id in block]
#                 self.examples.append(block)
#                 self.masks.append(torch.tensor(mask, dtype=int))    
                                            
#                 # Note that we are losing the last truncated example here for the sake of simplicity (no padding)
#                 # If your dataset is small, first you should loook for a bigger one :-) and second you
#                 # can change this behavior by adding (model specific) padding.
   

#             if os.path.exists(cached_features_file) and not overwrite_cache:
#                 start = time.time()
#                 with open(cached_features_file, "rb") as handle:
#                     self.examples = pickle.load(handle)
#                 logger.info(
#                     f"Loading features from cached file {cached_features_file} [took %.3f s]", time.time() - start
#                 )

#             else:

#                 start = time.time()
#                 with open(cached_features_file, "wb") as handle:
#                     pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)
#                 logger.info(
#                     "Saving features into cached file %s [took %.3f s]", cached_features_file, time.time() - start
#                 )
#                 with open("cached_mask", "wb") as handle:
#                     pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)


#     def __len__(self):
#         return len(self.examples)

#     def __getitem__(self, i) -> torch.Tensor:
#         return torch.tensor(self.examples[i], dtype=torch.long)

In [ ]:
# from transformers import TextDataset,DataCollatorForLanguageModeling

# def load_dataset(train_path,test_path,tokenizer):
#     train_dataset = CusTextDataset(
#           tokenizer=tokenizer,
#           file_path=train_path,
#           block_size=256)
     
#     test_dataset = CusTextDataset(
#           tokenizer=tokenizer,
#           file_path=test_path,
#           block_size=256)   
    
#     return train_dataset,test_dataset

# x_train_real, x_test_real = load_dataset('train_real.txt','test_real.txt',tokenizer)
# x_train_fake, x_test_fake = load_dataset('train_fake.txt','test_fake.txt',tokenizer)